In [7]:
import numpy as np
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'lib')))
from long_range_ising_rg import _Jprime, _linearise_Jprime, find_Jc

# Critical exponents

In [ ]:
compute_exponents(a=1, max_k=10, eps_matrix=1e-7)

In [ ]:
a_vals = np.linspace(0.80, 1.99, 20)
results = [compute_exponents(a, max_k=10, eps_matrix=1e-7) for a in a_vals]

yT = [r['y_T'] for r in results]
yH = [r['y_H'] for r in results]
alpha = [r['alpha'] for r in results]
beta = [r['beta']  for r in results]
gamma = [r['gamma'] for r in results]
delta = [r['delta'] for r in results]
nu = [r['nu']    for r in results]
eta = [r['eta']   for r in results]

In [ ]:
plt.figure(figsize=(8, 4))
#plt.suptitle('Critical exponents for 1d long-range Ising model (0.80 ≤ a ≤ 1.99)', fontsize=14)

plt.subplot(1, 2, 1)
plt.plot(a_vals, yT, '-', color='#ff7f0e')
plt.xlabel('a'); plt.ylabel(r'$y_T$')

plt.subplot(1, 2, 2)
plt.plot(a_vals, yH, '-', color='#d62728')
plt.xlabel('a'); plt.ylabel(r'$y_H$')

plt.tight_layout()
plt.savefig(f'../figures/critical_exponents_yT_yH.png')
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
#plt.suptitle('Critical exponents for 1d long-range Ising model (0.80 ≤ a ≤ 1.99)', fontsize=14)

plt.subplot(2, 3, 1)
plt.plot(a_vals, beta, '-', color='#ff7f0e', ms=4)
plt.xlabel('a'); plt.ylabel(r'$\beta$')

plt.subplot(2, 3, 2)
plt.plot(a_vals, delta, '-', color='#d62728', ms=4)
plt.xlabel('a'); plt.ylabel(r'$\delta$')

plt.subplot(2, 3, 3)
plt.plot(a_vals, alpha, '-', color='#1f77b4', ms=4)
plt.xlabel('a'); plt.ylabel(r'$\alpha$')

plt.subplot(2, 3, 4)
plt.plot(a_vals, gamma, '-', color='#2ca02c', ms=4)
plt.xlabel('a'); plt.ylabel(r'$\gamma$')

plt.subplot(2, 3, 5)
plt.plot(a_vals, nu, '-', color='#9467bd', ms=4)
plt.xlabel('a'); plt.ylabel(r'$\nu$')

plt.subplot(2, 3, 6)
plt.plot(a_vals, eta, '-', color='#8c564b', ms=4)
plt.xlabel('a'); plt.ylabel(r'$\eta$')

plt.tight_layout()
plt.savefig(f'../figures/critical_exponents.png')
plt.show()

# Critical temperature

In [ ]:
# Cell 1 – critical coupling
a = 1.3
Jc = find_Jc(a, max_k=1200, tol=1e-9)
print(f"a = {a:.3f}  →  Jc = {Jc:.10f}  (Tc = {1/Jc:.6f})")

# dH'/dH at Tc

In [ ]:
# Cell 2 – dH'/dH at the critical point
max_d = 3*1200 + 10
Jarr_crit = np.zeros(max_d+1, dtype=np.float64)
Jarr_crit[1:] = Jc * np.power(np.arange(1, max_d+1, dtype=np.float64), -a)

dh = dHdH(Jarr_crit, eps=1e-9)
yH = np.log(dh)/np.log(3.0)
print(f"dH'/dH = {dh:.6f}   →   y_H = {yH:.4f}")

## a vs dH'/dH

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- PARAMETERS ---
a_vals = np.linspace(0.8, 1.95, 30)   # avoid a=2.0 (first-order)
max_k = 1200
tol = 1e-9
eps = 1e-9

dh_vals = []
Jc_vals = []

print("Computing dH'/dH at criticality for a in [0.8, 1.95]...")
for a in a_vals:
    try:
        Jc = find_Jc(a, max_k=max_k, tol=tol)
        max_d = 3*max_k + 10
        Jarr = np.zeros(max_d+1, dtype=np.float64)
        Jarr[1:] = Jc * np.power(np.arange(1, max_d+1, dtype=np.float64), -a)
        
        dh = dHdH(Jarr, eps=eps)
        yH = np.log(dh)/np.log(3.0) if dh > 0 else np.nan
        
        dh_vals.append(dh)
        Jc_vals.append(Jc)
        print(f"a = {a:.3f} → Jc = {Jc:.6f}, dH'/dH = {dh:.6f}, y_H = {yH:.4f}")
    except:
        dh_vals.append(np.nan)
        Jc_vals.append(np.nan)

# --- PLOT ---
plt.figure(figsize=(7, 5))
valid = ~np.isnan(dh_vals)
plt.semilogy(np.array(a_vals)[valid], np.array(dh_vals)[valid], 
             'o-', color='tab:blue', markersize=5, linewidth=1.8, label="$\\partial H'/\\partial H$")

plt.xlabel('Interaction Range Exponent $a$')
plt.ylabel("$\\partial H'/\\partial H \\,|_{H=0}$")
plt.title('Field Renormalization at Criticality')
plt.grid(True, alpha=0.3, which='both', ls=':')
plt.legend()
plt.tight_layout()
plt.show()

# yT via Newton-Raphson

In [30]:
a = 1
max_k=8
md = 3 * max_k + 10
Jarr = np.zeros(md + 1, dtype=np.float64)
Jarr[1:] = 1 * np.power(np.arange(1, md + 1, dtype=np.float64), -a)
Jarr

array([0.        , 1.        , 0.5       , 0.33333333, 0.25      ,
       0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111,
       0.1       , 0.09090909, 0.08333333, 0.07692308, 0.07142857,
       0.06666667, 0.0625    , 0.05882353, 0.05555556, 0.05263158,
       0.05      , 0.04761905, 0.04545455, 0.04347826, 0.04166667,
       0.04      , 0.03846154, 0.03703704, 0.03571429, 0.03448276,
       0.03333333, 0.03225806, 0.03125   , 0.03030303, 0.02941176])

In [31]:
Jarr[1:max_k + 1]

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
       0.16666667, 0.14285714, 0.125     ])

In [ ]:
T = _linearise_Jprime(Jarr, max_k=max_k)
T

In [25]:
@njit
def _rg_step(Jarr, max_k):
    """Perform one RG transformation on a finite array of couplings."""
    rs = np.arange(1, max_k + 1, dtype=np.int64)
    Jp = np.empty(max_k, dtype=np.float64)
    for ri in range(max_k):
        Jp[ri] = _Jprime(3 * rs[ri] + 1, Jarr)   # r' = 1,…,max_k
    return Jp


def newton_fixed_point(a,
                       max_k=800,
                       tol=1e-12,
                       max_iter=30,
                       damp=1.0):
    """
    Newton-Raphson search for the RG fixed point at exponent a.
    Returns the fixed-point couplings J* (array of length max_k).
    """
    md = 3 * max_k + 10
    # ----- initial guess: critical Jc from bisection (already in the file) -----
    Jc = find_Jc(a, max_k=max_k, tol=1e-10)
    Jarr = np.zeros(md + 1, dtype=np.float64)
    Jarr[1:] = Jc * np.power(np.arange(1, md + 1, dtype=np.float64), -a)
    J = Jarr[1:max_k + 1].copy()                     # vector of N=max_k couplings

    for it in range(max_iter):
        # ---- RG image J' ----------------------------------------------------
        Jarr[1:max_k + 1] = J
        Jprime = _rg_step(Jarr, max_k)               # length max_k

        # ---- Jacobian T = ∂J'_i / ∂J_j ------------------------------------
        T = _linearise_Jprime(Jarr, max_k=max_k)     # (max_k × max_k)

        # ---- residual -------------------------------------------------------
        residual = Jprime - J

        # ---- solve linear system T δJ = residual ----------------------------
        from numpy.linalg import lstsq
        # --- inside newton_fixed_point(), replace the solve block ---
        # ---- solve linear system T δJ = residual (with regularization) ----
        try:
            # Option 1: Direct solve (fails on singular)
            # delta = np.linalg.solve(T, residual)
        
            # Option 2: Regularized least-squares (robust)
            delta, residuals, rank, s = lstsq(T, residual, rcond=1e-12)
            
            # Warn if rank deficient
            if rank < T.shape[0]:
                print(f"  [Newton] Warning: rank deficient Jacobian (rank={rank}/{T.shape[0]})")
        
        except np.linalg.LinAlgError:
            print("  [Newton] Direct solve failed, falling back to lstsq")
            delta, residuals, rank, s = lstsq(T, residual, rcond=1e-12)
        
        # Optional: add small Tikhonov regularization manually
        # reg = 1e-12 * np.eye(T.shape[0])
        # delta = np.linalg.solve(T.T @ T + reg, T.T @ residual)
            
        # ---- optional damping to improve convergence -----------------------
        J_new = J - damp * delta

        # ---- convergence test ---------------------------------------------
        err = np.max(np.abs(residual))
        if err < tol:
            return J_new

        J = J_new

    raise RuntimeError(f"Newton did not converge for a={a} after {max_iter} iterations")

In [27]:
# Example: fixed point at a = 1
a_val = 1.
J_fixed = newton_fixed_point(a_val, max_k=20, tol=1e-3
                            )
print("Fixed point (first 10 couplings):", J_fixed[:10])

  [Newton] Warning: rank deficient Jacobian (rank=10/20)
  [Newton] Warning: rank deficient Jacobian (rank=10/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (rank=8/20)
  [Newton] Warning: rank deficient Jacobian (r

RuntimeError: Newton did not converge for a=1.0 after 30 iterations